## AC Fireside Chat Group Activity

In this group activity, you will design the `best restaurant` you can think of.
- You have raised `X` amount of funding from VCs and angel investors.
- You may invest this cash on certain restaurant attributes.
- There are mandatory and wildcard attributes you may invest in:
    - `Mandatory attributes` are core to your business such as Food Quality, Portion Size, Service, Waiting Time, Aesthetics/Ambience
    - `Wildcard attributes` are optional investments that add unique advantages to your business
    - Wildcard attributes are purchased at a flat rate of `W`. Please ensure you have enough cash left upon selecting a wildcard.
- The `Affordability` attribute is automatically added and assumed to be inversely proportional to cash invested.
    
    
At the end of the activity, you will have to `pitch` why your restaurant is the best in town! Here's your pitch guide:
- Type of restaurant
- Attributes you invest in & why?

---

In [ ]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

attributes = sorted([
    'Food Quality',
    'Portion Size',
    'Service',
    'Aesthetics/Ambience'
])

wildcard_attributes = []
wildcard_attributes_all = sorted([
    'Good Location (SG based)',
    'Good Food Presentation',
    'Ease of Payment',
    'Good Food Variety',
    'Accommodates Dietary Requirement',
    'Good Capacity',
    'Good Freebies',
    'Marketing/Branding',
])

wildcard_selection = widgets.SelectMultiple(
    options=wildcard_attributes_all,
    # value=[],
    # rows=10,
    description='Wildcard:',
    disabled=False,
    style= {'description_width': 'initial'}
)

TOTAL_CASH = 70
CASH = TOTAL_CASH
WILDCARD_COST = 15

def create_slider(attr, value=0):
    int_slider = widgets.IntSlider(
        value=value,
        min=0,
        max=TOTAL_CASH,
        step=1,
        description=attr,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style= {'description_width': 'initial'}
    )
    int_slider.observe(on_value_change, names='value')
    int_slider.layout.width = '98%'
    return int_slider

def create_sliders():
    global sliders
    sliders = {}
    attributes_combined = attributes + list(wildcard_selection.value)

    # For each mandatory and wildcard attribute
    for attr in attributes_combined:
        # Create integer slider
        int_slider = create_slider(attr)
        sliders[attr] = int_slider
        
def reselect_sliders():
    global sliders, CASH, wildcard_selection
    
    with wildcard_prompt:
        wildcard_prompt.clear_output()
    
    attributes_to_remove = list(filter(
        lambda x: x not in attributes + list(wildcard_selection.value),
        sliders.keys()
    ))
    
    for attr in attributes_to_remove:
        CASH += sliders[attr].value
        sliders.pop(attr)
    
    attributes_to_add = list(filter(
        lambda x: x not in sliders.keys(),
        wildcard_selection.value
    ))
    
    for attr in attributes_to_add:
        if CASH >= WILDCARD_COST:
            int_slider = create_slider(attr=attr, value=WILDCARD_COST)
            CASH -= int_slider.value
            sliders[attr] = int_slider
        else:
            wildcard_selection.value = tuple(
                filter(
                    lambda x: x != attr,
                    wildcard_selection.value
                )
            )
            with wildcard_prompt:
                wildcard_prompt.clear_output()
                print('\033[91m' + f'Not enough cash to purchase `{attr}`.')
    
    with sliders_box:
        sliders_box.clear_output()
        display(widgets.VBox(
            list(
                get_mandatory_sliders(sliders).values()
            )
        ))
    
    with chart_output:
        chart_output.clear_output()
        build_dataframe()
        plot_dataframe()

def build_dataframe():
    global df
    df = pd.DataFrame(
        index=['Affordability'] + list(sliders.keys()),
        data=[CASH] + [round(slider.value, 0) for slider in sliders.values()],
        columns=['Value']
    )
    
def plot_dataframe():
    x_pos=np.arange(df.shape[0])
    
    plt.bar(
        x=x_pos,
        height=df['Value'],
        color=['green'] + ['cornflowerblue' for _ in attributes] + ['yellow' for _ in range(len(sliders) - len(attributes))]
    )
#     plt.ylim([0, TOTAL_CASH])
    plt.xticks(x_pos, df.index)
    plt.gcf().autofmt_xdate()
    plt.autoscale(
        enable=True,
        axis='x'
    )
    plt.title("My Restaurant's Stats")
    plt.ylabel('Stats')
    plt.show()

def on_value_change(change):
    global CASH, df, sliders
    
    if sum([slider.value for slider in sliders.values()]) > TOTAL_CASH:
        change['owner'].value = change['old'] + CASH
        
    CASH = TOTAL_CASH - sum([slider.value for slider in sliders.values()])
    
    with chart_output:
        chart_output.clear_output()
        build_dataframe()
        plot_dataframe()
        
def on_wildcard_selection_change(change):
    global sliders
    reselect_sliders()
    
    with sliders_box:
        sliders_box.clear_output()
        display(widgets.VBox(
            list(
                get_mandatory_sliders(sliders).values()
            )
        ))
        
def on_button_clicked(b):
    global CASH, slider
    wildcard_selection.value = ()
    reselect_sliders()
    
    for attr, slider in sliders.items():
        if slider.value > 0:
            CASH += slider.value
            slider.value = 0
            
    chart_output = out_temp
        
def get_mandatory_sliders(sliders):
    return dict(
            filter(
                lambda x: x[0] in attributes,
                sliders.items()
            )
        )

create_sliders()
wildcard_selection.observe(on_wildcard_selection_change, names='value')

sliders_box = widgets.Output()
with sliders_box:
    display(widgets.VBox(
        list(
                get_mandatory_sliders(sliders).values()
            )
    ))

remaining_cash_out = widgets.Output()
chart_output = widgets.Output()
with chart_output:
    build_dataframe()
    plot_dataframe()

wildcard_output = widgets.Output()
with wildcard_output:
    display(widgets.VBox([
        wildcard_selection
    ]))
wildcard_prompt = widgets.Output()

reset_button = widgets.Button(
    value=False,
    description='Reset',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset'
#     icon='check'
)
reset_button.on_click(on_button_clicked)

widgets.VBox([
    chart_output,
    sliders_box,
    wildcard_output,
    wildcard_prompt,
    reset_button
])